## Model training on AWS Sagemaker

Referencing [this notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_lstm_word_language_model/pytorch_rnn.ipynb)

In [10]:
import sagemaker
import boto3
import os
os.chdir('..')

In [2]:
sagemaker_session = sagemaker.Session()

In [4]:
bucket = sagemaker_session.default_bucket()
print(f"Bucket Name: {bucket}")

Bucket Name: sagemaker-us-east-1-594409465357


In [7]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName = 'mom-aws')['Role']['Arn']
print(f"Execution Role: {role}")

Couldn't call 'get_role' to get Role ARN from role name shaunkhoo to get Role path.


Execution Role: arn:aws:iam::594409465357:role/mom-aws


In [15]:
inputs = sagemaker_session.upload_data(path = "Data/Processed/Training/train-aws", 
                                       bucket = bucket, 
                                       key_prefix = prefix)

In [16]:
print(f"Inputs stored in: {inputs}")

Inputs stored in: s3://sagemaker-us-east-1-594409465357/sagemaker/ssoc-autocoder


In [22]:
env = {
    'SAGEMAKER_REQUIREMENTS': '../Notebooks/requirements.txt', # path relative to `source_dir` below.
}

In [34]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point = "train_aws.py",
    role = role,
    framework_version = "1.9.0",
    instance_count = 1,
    instance_type = "ml.p2.xlarge",
    source_dir = "ssoc_autocoder",
    py_version = "py3",
    env = env,
    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
    #                            bptt, dropout, tied, seed, log_interval
    hyperparameters = {"epochs": 1, "tied": True},
)

In [35]:
estimator.fit({"training": inputs})

2021-10-09 03:21:14 Starting - Starting the training job...
2021-10-09 03:21:37 Starting - Launching requested ML instancesProfilerReport-1633749695: InProgress
......
2021-10-09 03:22:46 Starting - Preparing the instances for training.........
2021-10-09 03:24:38 Downloading - Downloading input data...
2021-10-09 03:25:11 Training - Downloading the training image............
2021-10-09 03:27:22 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-09 03:27:23,719 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-10-09 03:27:23,745 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-09 03:27:23,749 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-09 03:27:24,112 sagemaker-containers INFO     Module default_user_module_name does not prov

In [ ]:
estimator.fit({"training": inputs})

2021-10-09 02:01:35 Starting - Starting the training job...
2021-10-09 02:01:37 Starting - Launching requested ML instancesProfilerReport-1633744916: InProgress
...
2021-10-09 02:02:35 Starting - Preparing the instances for training............
2021-10-09 02:04:54 Downloading - Downloading input data...
2021-10-09 02:05:34 Training - Downloading the training image.........
2021-10-09 02:07:15 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-09 02:07:03,451 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-10-09 02:07:03,476 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-09 02:07:03,480 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-09 02:07:03,818 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generat

UnexpectedStatusException: Error for Training job pytorch-training-2021-10-09-02-01-54-451: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_aws.py --epochs 1 --tied True"
Traceback (most recent call last):
  File "train_aws.py", line 16, in <module>
    from transformers import DistilBertModel, DistilBertTokenizer, DistilBertForSequenceClassification
ModuleNotFoundError: No module named 'transformers'